In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from pathlib import Path
from PIL import Image
from torchvision import transforms
import torch
import torch.optim as optim
import torch.nn as nn



In [2]:
DATA_DIR = "data"        # pasta com as duas classes
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4
DEVICE = "cuda" 

In [5]:
#Definição de parametros da RESNET50

from torchvision import transforms

vit_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # tamanho que o ViT espera
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [6]:
# Dividindo dataset em treino e validação
from Config import ImageDataset
from torch.utils.data import DataLoader, random_split

dataset = ImageDataset("data\DataSet", default_transform)  # pasta com as 2 classes

train_size = int(0.8 * len(dataset))
val_size   = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [7]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True)

In [8]:
# Modelo Vision Transformer (ViT-B/16)
num_classes = len(dataset.class_to_idx)
model = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)  # pré-treinada
model.heads.head = nn.Linear(model.heads.head.in_features, num_classes) # ajusta a última camada
model = model.to("cuda")  # GPU

# Critério de perda e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to C:\Users\Julio/.cache\torch\hub\checkpoints\vit_b_16-c867db91.pth


100.0%


In [9]:
from Config import train_and_validate

# Definir modelo, dataloaders, criterion, optimizer, etc.
MODEL_NAME = "VisualTransformer" # Nome do modelo
SAVE_PATH = f"{MODEL_NAME}_finetuned.pth"

train_and_validate(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=DEVICE,
    epochs=EPOCHS,
    save_path=SAVE_PATH
)

Epoch 1/10 | Train Loss: 0.2210 Acc: 0.9038 | Val Loss: 0.0879 Acc: 0.9684
Epoch 2/10 | Train Loss: 0.1024 Acc: 0.9605 | Val Loss: 0.0721 Acc: 0.9724
Epoch 3/10 | Train Loss: 0.0455 Acc: 0.9798 | Val Loss: 0.1234 Acc: 0.9625
Epoch 4/10 | Train Loss: 0.0413 Acc: 0.9872 | Val Loss: 0.0596 Acc: 0.9704
Epoch 5/10 | Train Loss: 0.0449 Acc: 0.9812 | Val Loss: 0.0595 Acc: 0.9763
Epoch 6/10 | Train Loss: 0.0181 Acc: 0.9931 | Val Loss: 0.0429 Acc: 0.9803
Epoch 7/10 | Train Loss: 0.0498 Acc: 0.9803 | Val Loss: 0.0667 Acc: 0.9684
Epoch 8/10 | Train Loss: 0.0403 Acc: 0.9837 | Val Loss: 0.0716 Acc: 0.9724
Epoch 9/10 | Train Loss: 0.0276 Acc: 0.9891 | Val Loss: 0.0922 Acc: 0.9744
Epoch 10/10 | Train Loss: 0.0392 Acc: 0.9877 | Val Loss: 0.0749 Acc: 0.9704
Modelo salvo em VisualTransformer_finetuned.pth
